In [32]:
print("hi")

hi


In [33]:
from langchain_groq import ChatGroq

In [34]:
from dotenv import load_dotenv

In [35]:
load_dotenv()

True

In [ ]:
llm=ChatGroq(model="deepseek-r1-distil-llama-70b")

In [ ]:
print(llm.invoke("What is the capital of France?").content)

In [36]:
# Quick credential check
import os
print('GOOGLE_API_KEY set?', bool(os.environ.get('GOOGLE_API_KEY')))

GOOGLE_API_KEY set? True


In [37]:
model=ChatGroq(model="qwen/qwen3-32b")

In [38]:
model.invoke("What is the capital of France?").content

"<think>\nOkay, the user is asking for the capital of France. Let me think. I know that France is a country in Europe. The capital... Hmm, I've heard Paris mentioned a lot. Is that correct? Let me confirm. Paris is a major city in France, home to the Eiffel Tower, the Louvre Museum, and the Seine River. Yeah, I think that's right. But wait, are there any other cities that could be the capital? Maybe Lyon or Marseille? No, I don't think so. Lyon is a big city but in the east, and Marseille is on the Mediterranean coast. I think Paris is definitely the capital. Let me just double-check my memory. France's government is based in Paris, right? The President's office is there. Yeah, I'm pretty confident the answer is Paris. I don't see any other cities being mentioned in this context. So the answer should be Paris.\n</think>\n\nThe capital of France is **Paris**. It is one of the most iconic cities in the world, renowned for landmarks such as the Eiffel Tower, the Louvre Museum, and the Sei

In [39]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [40]:
# Use the gemini embedding model identifier
embedding_model=GoogleGenerativeAIEmbeddings(model="gemini-embedding-001")

In [41]:
# Run a quick test and print the embedding length
vec = embedding_model.embed_query("What is the capital of France?")
print('embedding length:', len(vec))

GoogleGenerativeAIError: Error embedding content: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 

In [ ]:
embedding_model.embed_query("What is the capital of France?")

GoogleGenerativeAIError: Error embedding content: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 

In [ ]:
## 1. Data Injection


## RAG
## 1. Data Injection

In [ ]:
# install required packages for langchain document loaders and PDF parsing
#%pip install langchain langchain-community pypdf --quiet
from langchain_community.document_loaders import PyPDFLoader


In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os

In [ ]:
file_path=os.path.join(os.getcwd(), 'data', 'Medical_book.pdf')

In [ ]:
documents = PyPDFLoader(file_path).load()

In [ ]:
len(documents)

637

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

In [ ]:
docs = text_splitter.split_documents(documents)

In [ ]:
len(docs)

In [ ]:
docs[0].metadata

In [ ]:
docs[23].page_content

In [ ]:
# Prefer langchain import; fall back to langchain_community if necessary
try:
    from langchain.vectorstores import FAISS
except Exception:
    from langchain_community.vectorstores import FAISS

### Retrival from vector store

In [ ]:
vectorstore = FAISS.from_documents(docs, embedding_model)

In [ ]:
vectorstore.similarity_search("what is dermatology")

In [ ]:
relevent_doc=vectorstore.similarity_search("what is dermatology", k=2)

In [ ]:
relevent_doc[0].page_content

In [ ]:
relevent_doc[1].page_content

In [ ]:
retriver=vectorstore.as_retriever(search_kwargs={"k": 4})   

In [ ]:
retriver.invoke("what is dermatology")

## context = from vector database 
## question = user question 

In [ ]:
prompt_template = """ Answer he question based on context provoded below.
If the contect does not contain the answer, say you don't know.
context: {context}
question: {question}
answer:"""


In [ ]:
from langchain.prompts import PromptTemplate

In [ ]:
prompt=PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

In [ ]:
prompt

In [ ]:
from langchain_core.output_parsers import StrOutputParser

In [ ]:
parser=StrOutputParser()

In [ ]:
def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

In [ ]:
rag_chain=(
{"context": retriver|format_docs,"question": RunnablePassThrough()}
 | prompt
 | llm
 |StrOutputParser()
)

In [ ]:
rag_chain.invoke("what is cardialogy")